# Анализ базы данных книжного сервиса

После пандемии коронавируса компания купила крупный сервис для чтения книг по подписке.  В базе данных информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут нам провести анализ и сформулировать ценностное предложение для нового продукта.  

**Цель проекта:** проанализировать базу данных.  
Для достижения цели необходимо выполнить следующие **задачи**:  
1. Ознакомиться с данными и исследовать таблицы;  
2. Посчитать, сколько книг вышло после 1 января 2000 года;  
3. Для каждой книги посчитать количество обзоров и среднюю оценку;
4. Определить издательство, которое выпустило наибольшее число книг толще 50 страниц;  
5. Определить автора с самой высокой средней оценкой книг;  
6. Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.  


**Описание данных**

Таблица **books**  
Содержит данные о книгах:  
+ *book_id* — идентификатор книги;  
+ *author_id* — идентификатор автора;  
+ *title* — название книги;  
+ *num_pages* — количество страниц;  
+ *publication_date* — дата публикации книги;  
+ *publisher_id* — идентификатор издателя.  

Таблица **authors**  
Содержит данные об авторах:  
+ *author_id* — идентификатор автора;  
+ *author* — имя автора.  

Таблица **publishers**  
Содержит данные об издательствах:  
+ *publisher_id* — идентификатор издательства;  
+ *publisher* — название издательства;   

Таблица **ratings**  
Содержит данные о пользовательских оценках книг:  
+ *rating_id* — идентификатор оценки;  
+ *book_id* — идентификатор книги;  
+ *username* — имя пользователя, оставившего оценку;  
+ *rating* — оценка книги.  

Таблица **reviews**  
Содержит данные о пользовательских обзорах:  
+ *review_id* — идентификатор обзора;  
+ *book_id* — идентификатор книги;  
+ *username* — имя автора обзора;  
+ *text* — текст обзора.  

**Импорт библиотек**

In [1]:
import pandas as pd
from sqlalchemy import text, create_engine
import warnings
warnings.filterwarnings('ignore')

**Подключение к базе данных**

In [2]:
#устанавливаем параметры
db_config = {'user': 'praktikum_student', #имя пользователя
'pwd': 'Sdf4$2;d-d30pp', #пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, #порт подключения
'db': 'data-analyst-final-project-db'} #название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
#сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

## Исследование таблиц

In [3]:
#выведем первые пять строк таблицы books
query = '''
SELECT * 
FROM books 
LIMIT 5
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
#выведем первые пять строк таблицы authors
query = '''
SELECT * 
FROM authors 
LIMIT 5
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [5]:
#выведем первые пять строк таблицы publishers
query_3 = '''
SELECT * 
FROM publishers 
LIMIT 5
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [6]:
#выведем первые пять строк таблицы ratings
query = '''
SELECT * 
FROM ratings
LIMIT 5
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [7]:
#выведем первые пять строк таблицы reviews
query = '''
SELECT * 
FROM reviews
LIMIT 5
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Для анализа у нас есть пять таблиц, данные считываются корректно. Можем переходить к следующим запросам.

## Посчитаем, сколько книг вышло после 1 января 2000 года

In [8]:
query = '''
SELECT COUNT(book_id)
FROM books 
WHERE publication_date > '2000-01-01' 
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,819


После 1 января 2000 года вышло 819 книг.

## Для каждой книги посчитаем количество обзоров и среднюю оценку

In [9]:
query = '''
SELECT b.book_id,
       b.title, 
       COUNT(DISTINCT rev.review_id) AS cnt_reviews,
       ROUND(AVG(rat.rating),2) AS avg_rating                 
FROM books AS b
LEFT JOIN reviews AS rev ON b.book_id=rev.book_id
LEFT JOIN ratings AS rat ON b.book_id=rat.book_id
GROUP BY b.book_id
ORDER BY cnt_reviews DESC

'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,title,cnt_reviews,avg_rating
0,948,Twilight (Twilight #1),7,3.66
1,963,Water for Elephants,6,3.98
2,734,The Glass Castle,6,4.21
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.41
4,695,The Curious Incident of the Dog in the Night-Time,6,4.08
...,...,...,...,...
995,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.67
996,808,The Natural Way to Draw,0,3.00
997,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.00
998,221,Essential Tales and Poems,0,4.00


Количество обзоров и средняя оценка для каждой книги посчитаны. Больше всего обзоров (7) у книги Twilight (Twilight #1) со средней оценкой 3.66.

## Определим издательство, которое выпустило наибольшее число книг толще 50 страниц (так исключим из анализа брошюры)

In [10]:
query = '''
SELECT b.publisher_id,
       pub.publisher,
       COUNT(b.book_id) AS cnt_books
FROM books AS b
LEFT JOIN publishers AS pub ON b.publisher_id=pub.publisher_id
WHERE b.num_pages > 50
GROUP BY b.publisher_id, pub.publisher_id
ORDER BY cnt_books DESC
LIMIT 1 
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher,cnt_books
0,212,Penguin Books,42


Больше всего книг выпустило издтательство Penguin Books.

## Определим автора с самой высокой средней оценкой книг (учтем только книги с 50 и более оценками)

In [11]:
query = '''
SELECT author,
       ROUND(AVG(rating)) AS avg_rating
FROM books AS b
LEFT JOIN authors AS a ON b.author_id=a.author_id
LEFT JOIN ratings AS rat ON b.book_id=rat.book_id
WHERE b.book_id IN
                        (
                        SELECT book_id
                        FROM ratings
                        GROUP BY book_id
                        HAVING COUNT(rating_id) > 50
                        )
GROUP BY author
ORDER BY avg_rating DESC
LIMIT 1


'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.0


Автор с самой высокой средней оценкой книг (среди книг с 50 и более оценками) - J.K. Rowling/Mary GrandPré, средняя оценка 4.0.

## Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [12]:
query = '''
WITH ur AS 
          (
          SELECT COUNT(review_id) FROM reviews
          WHERE username in 
                            (
                            SELECT username 
                            FROM ratings
                            GROUP BY username
                            HAVING COUNT(rating_id) > 48
                            )
         GROUP BY username
         )
SELECT AVG(count) as average_user_reviews_count
FROM ur
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,average_user_reviews_count
0,24.0


Cреднее количество обзоров от пользователей, которые поставили больше 48 оценок составляет 24 обзора.

## Выводы

1. После 1 января 2000 года вышло 819 книг.  
2. Количество обзоров и средняя оценка для каждой книги посчитаны. Больше всего обзоров (7) у книги Twilight (Twilight #1) со средней оценкой 3.66.  
3. Больше всего книг выпустило издтательство Penguin Books.  
4. Автор с самой высокой средней оценкой книг (среди книг с 50 и более оценками) - J.K. Rowling/Mary GrandPré, средняя оценка 4.0.  
5. Cреднее количество обзоров от пользователей, которые поставили больше 48 оценок составляет 24 обзора.